In [1]:
import praw
import pandas as pd
import seaborn as sns
import numpy as np

In [2]:
import re    # RegEx for removing non-letter characters
import nltk  #natural language processing
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem.porter import *

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
from spacy_langdetect import LanguageDetector
import spacy

In [44]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
reddit = praw.Reddit(client_id='TlPT-9dolaABWzzGIxim8Q', client_secret='KcSNwe14od5UfnHs-Ps5mBfdPYm6RA', \
                     user_agent='Jenifer Vivar', check_for_async=False)


In [5]:
sub = ['Askreddit']  # make a list of subreddits you want to scrape the data from

for s in sub:
    subreddit = reddit.subreddit(s) 

In [6]:
import spacy
from spacy.language import Language
from spacy_langdetect import LanguageDetector

def get_lang_detector(nlp, name):
    return LanguageDetector()

nlp = spacy.load("en_core_web_sm")
Language.factory("language_detector", func=get_lang_detector)
nlp.add_pipe('language_detector', last=True)
text = 'Hola que tal, como estas'
doc = nlp(text)
print(doc._.language)

{'language': 'es', 'score': 0.9999946315527208}


In [7]:
text = 'Hola que tal, como estas'
doc = nlp(text)
print(doc._.language, doc._.language["language"])

{'language': 'es', 'score': 0.999994785474744} es


In [8]:
def tweet_to_words(tweet):
    ''' Convert tweet text into a sequence of words '''
    
    # convert to lowercase
    text = tweet.lower()
    # remove non letters
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    # tokenize
    words = text.split()
    # remove stopwords
    words = [w for w in words if w not in stopwords.words("english")]
    # apply stemming
    words = [PorterStemmer().stem(w) for w in words]
    # return list
    return words

In [ ]:

query = ['moral', "ethical", "morality", "rigth", "fair", ]
dfs_comments = []
dfs_data = []
for item in query:

  post_dict = {
      "title" : [],   #title of the post
      "score" : [],   # score of the post
      "id" : [],      # unique id of the post
      "url" : [],     #url of the post
      "comms_num": [],   #the number of comments on the post
      "created" : [],  #timestamp of the post
      "body" : []         # the descriptionof post
  }
  comments_dict = {
      "comment_id" : [],      #unique comm id
      "comment_parent_id" : [],   # comment parent id
      "comment_body" : [],   # text in comment
      "comment_link_id" : [],
      "comment_important_words":[] #link to the comment
       
  }
  
  for submission in subreddit.search(query, sort = "top",limit = 10):
      post_dict["title"].append(submission.title)
      post_dict["score"].append(submission.score)
      post_dict["id"].append(submission.id)
      post_dict["url"].append(submission.url)
      post_dict["comms_num"].append(submission.num_comments)
      post_dict["created"].append(submission.created)
      post_dict["body"].append(submission.selftext)
      
      ##### Acessing comments on the post
      submission.comments.replace_more(limit = 1)
      for comment in submission.comments.list():
        comment_ = tweet_to_words(comment.body)
        comments_dict["comment_id"].append(comment.id)
        comments_dict["comment_parent_id"].append(comment.parent_id)
        comments_dict["comment_body"].append(comment.body)
        comments_dict["comment_important_words"].append(comment_)
        comments_dict["comment_link_id"].append(comment.link_id)
        

  post_comments = pd.DataFrame(comments_dict)
  dfs_comments.append(post_comments)
  #post_comments.to_csv(s+"_comments_"+ item +"subreddit.csv")
  post_data = pd.DataFrame(post_dict)
  dfs_data.append(post_data)
  #post_data.to_csv(s+"_"+ item +"subreddit.csv")



In [30]:
df_concat_comments = pd.concat(dfs_comments, axis=0)
df_concat_comments.head()
#df_concat_comments.to_csv("/Users/jenifervivar/Documents/GitHub/deep-learning-final-project-yelp_reviews_classification/data/external/comments_subreddit.csv")

,comment_id,comment_parent_id,comment_body,comment_link_id,comment_important_words
0,fspkqqw,t3_gvj9a9,Im really starting to think 2020 is now offici...,t3_gvj9a9,"[im, realli, start, think, 2020, offici, year,..."
1,fspcygb,t3_gvj9a9,"Regardless of your stance, political or otherw...",t3_gvj9a9,"[regardless, stanc, polit, otherwis, think, ag..."
2,fspei0f,t3_gvj9a9,We need an end goal. \n\nIt's not enough to sa...,t3_gvj9a9,"[need, end, goal, enough, say, want, chang, ne..."
3,fsp1wiu,t3_gvj9a9,I'm not American but I can see this cycle happ...,t3_gvj9a9,"[american, see, cycl, happen, someth, happen, ..."
4,fspcczo,t3_gvj9a9,I saw a live stream from Charleston where they...,t3_gvj9a9,"[saw, live, stream, charleston, organ, get, re..."


In [51]:
df_concat_comments["comment_body"].iloc[7]

"I just have to say I've seen some very negative things about the whole situation on some different subreddits, even my own city' subreddit which I thought would be fully supportive. Turns out they are the complete opposite.\n\nBut, there's been a whole lot of love, caring, and support coming from MOST subreddits and I truly appreciate it. It means the world honestly."

In [48]:
len(df_concat_comments)

29255

In [27]:
df_metadata = pd.concat(dfs_data, axis=0)
df_metadata.head()
#df_metadata.to_csv("/Users/jenifervivar/Documents/GitHub/deep-learning-final-project-yelp_reviews_classification/data/external/metadata_subreddit.csv")

,title,score,id,url,comms_num,created,body
0,I can’t breathe. Black lives matter.,96763,gvj9a9,https://www.reddit.com/r/AskReddit/comments/gv...,6748,1.591143e+09,As the gap of the political divide in our worl...
1,How do you feel about ’how would you feel?’ po...,95639,mje9y7,https://www.reddit.com/r/AskReddit/comments/mj...,3475,1.617475e+09,We’ve gotten a lot of feedback on this one. A ...
2,"Where can you find fair, honest and reasonably...",63136,igt38u,https://www.reddit.com/r/AskReddit/comments/ig...,13012,1.598420e+09,
3,"Reddit, Can we have a reddit job fair?",37130,29oykm,https://www.reddit.com/r/AskReddit/comments/29...,22180,1.404343e+09,"Hi Reddit,\nI (and probably many others too) d..."
4,"Mother-In-Laws of Reddit, in the interest of f...",15964,lfibsb,https://www.reddit.com/r/AskReddit/comments/lf...,1782,1.612809e+09,


In [49]:
df_metadata["title"].value_counts()

I can’t breathe. Black lives matter.                                                                                                                          5
How do you feel about ’how would you feel?’ posts being banned?                                                                                               5
Where can you find fair, honest and reasonably unbiased opinions on world news and politics?                                                                  5
Reddit, Can we have a reddit job fair?                                                                                                                        5
Mother-In-Laws of Reddit, in the interest of fairness, what are your crazy daughter-in-law stories?                                                           5
You die. Death himself however says if you can beat him at a fair game of your choice, you get a second chance at life. What game do you challenge him to?    5
Who is the best example of someone that 

In [41]:
for title in df_metadata["title"]:
  print(title)

I can’t breathe. Black lives matter.
How do you feel about ’how would you feel?’ posts being banned?
Where can you find fair, honest and reasonably unbiased opinions on world news and politics?
Reddit, Can we have a reddit job fair?
Mother-In-Laws of Reddit, in the interest of fairness, what are your crazy daughter-in-law stories?
You die. Death himself however says if you can beat him at a fair game of your choice, you get a second chance at life. What game do you challenge him to?
Who is the best example of someone that you're not sure if they are a complete idiot or a genius?
[UPDATE] I was requested to compile all of the real-life cheats into an easy to read list.
Reddit, what's a fairly unknown yet super interesting subreddit that will pass the time?
Reddit, what is the most fucked up way you got back at someone that wronged you. *Throwaways encouraged*
I can’t breathe. Black lives matter.
How do you feel about ’how would you feel?’ posts being banned?
Where can you find fair, hon

In [36]:
len(post_comments)

593

In [45]:
post_data

,title,score,id,url,comms_num,created,body
0,what scientific experiment would you run if mo...,73956,e31v9o,https://www.reddit.com/r/AskReddit/comments/e3...,19529,1.574965e+09,


In [49]:
dfs_data[1]

,title,score,id,url,comms_num,created,body
0,what scientific experiment would you run if mo...,73954,e31v9o,https://www.reddit.com/r/AskReddit/comments/e3...,19529,1.574965e+09,


In [ ]:

query = ['moral',"Morality", "ethical"]
dfs_comments = []
dfs_data = []
for item in query:

  post_dict = {
      "title" : [],   #title of the post
      "score" : [],   # score of the post
      "id" : [],      # unique id of the post
      "url" : [],     #url of the post
      "comms_num": [],   #the number of comments on the post
      "created" : [],  #timestamp of the post
      "body" : []         # the descriptionof post
  }
  comments_dict = {
      "comment_id" : [],      #unique comm id
      "comment_parent_id" : [],   # comment parent id
      "comment_body" : [],   # text in comment
      "comment_link_id" : [],
      "comment_important_words":[]  #link to the comment
  }
  result 
  for submission in subreddit.search(query,sort = "top",limit = 3000):
      post_dict["title"].append(submission.title)
      post_dict["score"].append(submission.score)
      post_dict["id"].append(submission.id)
      post_dict["url"].append(submission.url)
      post_dict["comms_num"].append(submission.num_comments)
      post_dict["created"].append(submission.created)
      post_dict["body"].append(submission.selftext)
      
      ##### Acessing comments on the post
      submission.comments.replace_more(limit = 1)
      for comment in submission.comments.list():
        comment_ = tweet_to_words(comment.body)
        comments_dict["comment_id"].append(comment.id)
        comments_dict["comment_parent_id"].append(comment.parent_id)
        comments_dict["comment_body"].append(comment.body)
        comments_dict["comment_important_words"].append(comment_)
        comments_dict["comment_link_id"].append(comment.link_id)
  
  post_comments = pd.DataFrame(comments_dict)
  dfs_comments.append(post_comments)
  #post_comments.to_csv(s+"_comments_"+ item +"subreddit.csv")
  post_data = pd.DataFrame(post_dict)
  dfs_data.append(post_data)
  #post_data.to_csv(s+"_"+ item +"subreddit.csv")

